In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [3]:
page_url = 'https://www.novibet.gr/en/sports'
driver.get(page_url)
cookies = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'acceptCookies_button'))).click()

In [4]:
# Close Log in pop up window
x_button = driver.find_element(By.CSS_SELECTOR, '[data-cy="closeBtn"]').click()

## 1. Football

In [5]:
# Click Daily Coupon (Football)
dayly_coupon_button = driver.find_element(By.CSS_SELECTOR, 'a.ng-star-inserted[title="Daily coupon"]').click()

In [6]:
# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
# By deafult is the football first
daily_coupon_body = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract footbal text
football_string = daily_coupon_body.text

In [7]:
# Create list from the initial string
initial_list = football_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First','SO']

football_list = [x for x in initial_list if x not in remove_elements]

### I want to find a way to increace the size of i if i insert elements

# Use padding for 'Markets are not available'
index = 0
while index < len(football_list):
    if football_list[index] == 'Markets are not available':
        football_list.insert(index+1, 'No_market')
        football_list.insert(index+2, 'No_market')
        football_list.insert(index+3, 'No_market')
        index += 4
    else:
        index += 1

In [8]:
# Create sublists based on Championship
championship = [x for x in football_list if ' - ' in x]
index_championship = [i for i,x in enumerate(football_list) if ' - ' in x]
sublists_championships = [football_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(football_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [9]:
# Create dataframe 
df_football = pd.DataFrame(columns = ['championship','team1','team2','time','1','odds_1','X','odds_X','2','odds_2',
                             'Over','odds_over','Under','odds_under','GG','odds_gg','NG','odds_ng'])

# Loop through all championships
for j in range(len(sublists_championships)):
     
    team1_lst, team2_lst = [],[]
    time_lst = []
    one_lst, one_odds_lst, x_lst, x_odds_lst, two_lst, two_odds_lst = [],[],[],[],[],[]
    over_lst, over_odds_lst, under_lst, under_odds_lst = [],[],[],[]
    gg_lst, gg_odds_lst, ng_lst, ng_odds_lst = [],[],[],[]

    # Create list with only teams and odds / exclude the championship at the beginning
    teams_only_lst = sublists_championships[j][1:]
    championship_lst = [sublists_championships[j][0] for _ in range(len(teams_only_lst) // 17)]
    

    i = 0
    while i < len(teams_only_lst):
        team1_lst.append(teams_only_lst[i])
        team2_lst.append(teams_only_lst[i+1])
        time_lst.append(teams_only_lst[i+2])
        one_lst.append(teams_only_lst[i+3])
        one_odds_lst.append(teams_only_lst[i+4])
        x_lst.append(teams_only_lst[i+5])
        x_odds_lst.append(teams_only_lst[i+6])
        two_lst.append(teams_only_lst[i+7])
        two_odds_lst.append(teams_only_lst[i+8])
        over_lst.append(teams_only_lst[i+9])
        over_odds_lst.append(teams_only_lst[i+10])
        under_lst.append(teams_only_lst[i+11])
        under_odds_lst.append(teams_only_lst[i+12])
        gg_lst.append(teams_only_lst[i+13])
        gg_odds_lst.append(teams_only_lst[i+14])
        ng_lst.append(teams_only_lst[i+15])
        ng_odds_lst.append(teams_only_lst[i+16])
        i = i + 17

    # Create the dataframe
    df = pd.DataFrame({
        'championship': championship_lst,
        'team1': team1_lst,
        'team2': team2_lst,
        'time': time_lst,
        '1': one_lst,
        'odds_1': one_odds_lst,
        'X': x_lst,
        'odds_X': x_odds_lst,
        '2': two_lst,
        'odds_2': two_odds_lst,
        'Over': over_lst,
        'odds_over': over_odds_lst,
        'Under': under_lst,
        'odds_under': under_odds_lst,
        'GG': gg_lst,
        'odds_gg': gg_odds_lst,
        'NG': ng_lst,
        'odds_ng': ng_odds_lst
    })

    df_football = df_football.append(df, ignore_index = True)

In [10]:
df_football

,championship,team1,team2,time,1,odds_1,X,odds_X,2,odds_2,Over,odds_over,Under,odds_under,GG,odds_gg,NG,odds_ng
0,Europe - EURO 2024 - Qualifying,Bosnia & Herzegovina,Iceland,21:45,1,1.75,X,3.65,2,4.90,O 2.5,2.20,U 2.5,1.67,GG,2.10,NG,1.71
1,Europe - EURO 2024 - Qualifying,Denmark,Finland,21:45,1,1.35,X,4.75,2,10.00,O 2.5,2.01,U 2.5,1.81,GG,2.50,NG,1.51
2,Europe - EURO 2024 - Qualifying,Italy,England,21:45,1,3.05,X,3.20,2,2.74,O 2.5,2.45,U 2.5,1.55,GG,2.03,NG,1.76
3,Europe - EURO 2024 - Qualifying,North Macedonia,Malta,21:45,1,1.36,X,4.75,2,9.75,O 2.5,1.97,U 2.5,1.85,GG,2.45,NG,1.54
4,Europe - EURO 2024 - Qualifying,Portugal,Liechtenstein,21:45,1,1.001,X,41.00,2,61.00,O 6.5,2.04,U 6.5,1.75,Markets are not available,No_market,No_market,No_market
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Central America - CONCACAF Nations League,Barbados,Cuba,Fri 01:00,1,7.00,X,4.55,2,1.41,O 2.5,1.70,U 2.5,2.10,GG,1.92,NG,1.82
132,Central America - CONCACAF Nations League,Guadeloupe,Antigua & Barbuda,Fri 01:00,1,1.29,X,4.95,2,8.70,O 2.5,1.68,U 2.5,2.10,GG,2.15,NG,1.67
133,Central America - CONCACAF Nations League,Suriname,Mexico,Fri 02:00,1,18.00,X,7.60,2,1.15,O 3.5,1.91,U 3.5,1.87,GG,2.15,NG,1.66
134,Bahrain - Federations Cup,Hidd Club,Al Khalidiyah,in 12',1,4.25,X,3.85,2,1.68,O 2.5,1.87,U 2.5,1.87,GG,1.87,NG,1.87


## 2. Basket

In [11]:
# Click the basketball button
basketball_button = driver.find_element(By.CSS_SELECTOR, '[class = "svgImage default BASKETBALL_GAME medium"]').click()

# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
daily_coupon_body = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract basketball text
basketball_string = daily_coupon_body.text

In [12]:
# Create list from the initial string
initial_list = basketball_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First','SO']

basketball_list = [x for x in initial_list if x not in remove_elements]

# Use padding for 'Markets are not available'
index = 0
while index < len(basketball_list):
    if basketball_list[index] == 'Markets are not available':
        basketball_list.insert(index+1, 'No_market')
        basketball_list.insert(index+2, 'No_market')
        basketball_list.insert(index+3, 'No_market')
        index += 4
    else:
        index += 1

In [13]:
# Create sublists based on Championship
championship = [x for x in basketball_list if ' - ' in x]
index_championship = [i for i,x in enumerate(basketball_list) if ' - ' in x]
sublists_championships = [basketball_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(basketball_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [16]:
# Create dataframe 
df_basketball = pd.DataFrame(columns = ['championship','team1','team2','time','win_1','odds_win_1','win_2','odds_win_2',
                                        'Over','odds_over','Under','odds_under','1','odds_1','2','odds_2'])

# Loop through all championships
for j in range(len(sublists_championships)):
     
    team1_lst, team2_lst = [],[]
    time_lst = []
    win_1_lst, win_1_odds_lst, win_2_lst, win_2_odds_lst = [],[],[],[]
    over_lst, over_odds_lst, under_lst, under_odds_lst = [],[],[],[]
    one_lst, one_odds_lst, two_lst, two_odds_lst = [],[],[],[]

    # Create list with only teams and odds / exclude the championship at the beginning
    teams_only_lst = sublists_championships[j][1:]
    championship_lst = [sublists_championships[j][0] for _ in range(len(teams_only_lst) // 15)]
    

    i = 0
    while i < len(teams_only_lst):
        team1_lst.append(teams_only_lst[i])
        team2_lst.append(teams_only_lst[i+1])
        time_lst.append(teams_only_lst[i+2])
        win_1_lst.append(teams_only_lst[i+3])
        win_1_odds_lst.append(teams_only_lst[i+4])
        win_2_lst.append(teams_only_lst[i+5])
        win_2_odds_lst.append(teams_only_lst[i+6])
        over_lst.append(teams_only_lst[i+7])
        over_odds_lst.append(teams_only_lst[i+8])
        under_lst.append(teams_only_lst[i+9])
        under_odds_lst.append(teams_only_lst[i+10])
        one_lst.append(teams_only_lst[i+11])
        one_odds_lst.append(teams_only_lst[i+12])
        two_lst.append(teams_only_lst[i+13])
        two_odds_lst.append(teams_only_lst[i+14])
        i = i + 15

    # Create the dataframe
    df = pd.DataFrame({
        'championship': championship_lst,
        'team1': team1_lst,
        'team2': team2_lst,
        'time': time_lst,
        'win_1': win_1_lst,
        'odds_win_1': win_1_odds_lst,
        'win_2': win_2_lst,
        'odds_win_2': win_2_odds_lst,
        'Over': over_lst,
        'odds_over': over_odds_lst,
        'Under': under_lst,
        'odds_under': under_odds_lst,
        '1': one_lst,
        'odds_1': one_odds_lst,
        '2': two_lst,
        'odds_2': two_odds_lst
    })

    df_basketball = df_basketball.append(df, ignore_index = True)

In [17]:
df_basketball

,championship,team1,team2,time,win_1,odds_win_1,win_2,odds_win_2,Over,odds_over,Under,odds_under,1,odds_1,2,odds_2
0,USA - NBA,ORL Magic,NY Knicks,Fri 01:00,+4.5,1.88,-4.5,1.93,O 228.5,1.90,U 228.5,1.91,1,2.50,2,1.56
1,USA - NBA,BKN Nets,CLE Cavaliers,Fri 01:30,+4.5,1.87,-4.5,1.95,O 218.5,1.88,U 218.5,1.93,1,2.50,2,1.57
2,USA - NBA,NO Pelicans,CHA Hornets,Fri 02:00,-8.5,1.86,+8.5,1.96,O 225.5,1.90,U 225.5,1.91,1,1.27,2,3.95
3,USA - NBA,LA Clippers,OKC Thunder,Fri 04:30,-4.5,1.93,+4.5,1.88,O 232.5,1.91,U 232.5,1.90,1,1.60,2,2.65
4,Europe - Euroleague,Alba Berlin,Anadolu Efes,in 26',+8.5,1.97,-8.5,1.84,O 166.5,1.94,U 166.5,1.87,1,4.80,2,1.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,USA - NIT (W),Bowling Green Falcons (W),Memphis Tigers (W),Fri 00:00,-4.5,1.87,+4.5,1.87,O 136.5,1.87,U 136.5,1.87,1,1.49,2,2.50
86,USA - NIT (W),Clemson Tigers (W),Florida Gators (W),Fri 01:00,-6.5,1.86,+6.5,1.88,O 134.5,1.87,U 134.5,1.87,1,1.34,2,3.10
87,USA - NIT (W),Harvard Crimson (W),Rhode Island Rams (W),Fri 01:00,-4.5,1.88,+4.5,1.86,O 131.5,1.87,U 131.5,1.87,1,1.50,2,2.45
88,USA - NIT (W),Kansas Jayhawks (W),Nebraska Corhuskers (W),Fri 01:30,-4.5,1.87,+4.5,1.87,O 137.5,1.87,U 137.5,1.87,1,1.49,2,2.50


## 3. Tennis

In [18]:
# Click the Tennis button
tennis_button = driver.find_element(By.CSS_SELECTOR, '[class = "svgImage default TENNIS_SINGLES_MATCH medium"]').click()

# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
daily_coupon_body = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract basketball text
tennis_string = daily_coupon_body.text

In [19]:
# Create list from the initial string
initial_list = tennis_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First','SO']

tennis_list = [x for x in initial_list if x not in remove_elements]

# Use padding for 'Markets are not available'
index = 0
while index < len(tennis_list):
    if tennis_list[index] == 'Markets are not available':
        tennis_list.insert(index+1, 'No_market')
        tennis_list.insert(index+2, 'No_market')
        tennis_list.insert(index+3, 'No_market')
        index += 4
    else:
        index += 1

In [20]:
# Create sublists based on Championship
championship = [x for x in tennis_list if ' - ' in x]
index_championship = [i for i,x in enumerate(tennis_list) if ' - ' in x]
sublists_championships = [tennis_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(tennis_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [23]:
# Create dataframe 
df_tennis = pd.DataFrame(columns = ['championship','player1','player2','time','1','odds_1','2','odds_2',
                                    'Over','odds_over','Under','odds_under','win_1','odds_win_1','win_2','odds_win_2',])

# Loop through all championships
for j in range(len(sublists_championships)):
     
    player1_lst, player2_lst = [],[]
    time_lst = []
    one_lst, one_odds_lst, two_lst, two_odds_lst = [],[],[],[]
    over_lst, over_odds_lst, under_lst, under_odds_lst = [],[],[],[]
    win_1_lst, win_1_odds_lst, win_2_lst, win_2_odds_lst = [],[],[],[]

    # Create list with only teams and odds / exclude the championship at the beginning
    teams_only_lst = sublists_championships[j][1:]
    championship_lst = [sublists_championships[j][0] for _ in range(len(teams_only_lst) // 15)]
    

    i = 0
    while i < len(teams_only_lst):
        player1_lst.append(teams_only_lst[i])
        player2_lst.append(teams_only_lst[i+1])
        time_lst.append(teams_only_lst[i+2])
        one_lst.append(teams_only_lst[i+3])
        one_odds_lst.append(teams_only_lst[i+4])
        two_lst.append(teams_only_lst[i+5])
        two_odds_lst.append(teams_only_lst[i+6])
        over_lst.append(teams_only_lst[i+7])
        over_odds_lst.append(teams_only_lst[i+8])
        under_lst.append(teams_only_lst[i+9])
        under_odds_lst.append(teams_only_lst[i+10])
        win_1_lst.append(teams_only_lst[i+11])
        win_1_odds_lst.append(teams_only_lst[i+12])
        win_2_lst.append(teams_only_lst[i+13])
        win_2_odds_lst.append(teams_only_lst[i+14])

        i = i + 15

    # Create the dataframe
    df = pd.DataFrame({
        'championship': championship_lst,
        'player1': player1_lst,
        'player2': player2_lst,
        'time': time_lst,
        '1': one_lst,
        'odds_1': one_odds_lst,
        '2': two_lst,
        'odds_2': two_odds_lst,
        'Over': over_lst,
        'odds_over': over_odds_lst,
        'Under': under_lst,
        'odds_under': under_odds_lst,
        'win_1': win_1_lst,
        'odds_win_1': win_1_odds_lst,
        'win_2': win_2_lst,
        'odds_win_2': win_2_odds_lst

    })

    df_tennis = df_tennis.append(df, ignore_index = True)

In [24]:
df_tennis

,championship,player1,player2,time,1,odds_1,2,odds_2,Over,odds_over,Under,odds_under,win_1,odds_win_1,win_2,odds_win_2
0,World - ATP Miami,Daniel Elahi Galan Riveros,Mackenzie McDonald,in 38',1,6.00,2,1.13,O 20.5,2.00,U 20.5,1.72,+5.5,1.81,-5.5,1.90
1,World - ATP Miami,Yosuke Watanuki,Benoit Paire,in 53',1,1.60,2,2.35,O 22.5,1.87,U 22.5,1.83,-2.5,1.90,+2.5,1.81
2,World - ATP Miami,Thiago Moura Monteiro,Jason Kubler,22:20,1,3.05,2,1.38,O 22.5,1.91,U 22.5,1.80,+3.5,1.85,-3.5,1.85
3,World - ATP Miami,Richard Gasquet,Christopher O Connell,22:30,1,1.78,2,2.05,O 22.5,1.79,U 22.5,1.92,-0.5,1.81,+0.5,1.90
4,World - ATP Miami,Gregoire Barrere,Roman Safiullin,23:35,1,2.27,2,1.64,O 22.5,1.84,U 22.5,1.86,+2.5,1.80,-2.5,1.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Asia - ITF Indonesia Doubles (W),MA Y / Uchijima M,Gumulya B / Rompies J,Fri 08:30,1,1.49,2,2.50,Markets are not available,No_market,No_market,No_market,Markets are not available,No_market,No_market,No_market
184,ЅRL - SRL Spring Invitational Nukus,Carlos Alcaraz (Srl),Alexander Zverev (Srl),Fri 10:00,1,1.62,2,2.15,O 22.5,1.81,U 22.5,1.90,Markets are not available,No_market,No_market,No_market
185,ЅRL - SRL Spring Invitational Nukus,Rafael Nadal (Srl),Casper Ruud (Srl),Fri 12:00,1,1.27,2,3.40,O 21.5,1.74,U 21.5,1.98,Markets are not available,No_market,No_market,No_market
186,ЅRL (W) - SRL Spring Invitational Nukus (W),Jelena Ostapenko (Srl),Liudmila Samsonova (Srl),Fri 10:00,1,1.79,2,1.92,O 21.5,1.90,U 21.5,1.81,Markets are not available,No_market,No_market,No_market
